## Lab 5 - Interactive Plotting

In [57]:
import pandas as pd

from bokeh.plotting import figure, show 
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.models.widgets import Panel, Tabs
from ipywidgets import interact
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

Documentation: https://docs.bokeh.org/en/0.8.2/docs/user_guide/intro.html


* figure - refers to the entire plot
* glyph -  basic visual building blocks, e.g. lines, rectangles, circles, etc.
* output_file() - specifies how to output the plot
* show() - displays the plot

### Importing & merging data

In [18]:
# Download reported crime data from 2019

def get_data_chicago(id):
    '''
    Connect to the chicago data portal API and returns a dataframe
    '''
    
    url = f'https://data.cityofchicago.org/api/views/{id}/rows.csv?accessType=DOWNLOAD'
    df = pd.read_csv(url)

    
    return df


def crime_data():
    ''' 
    Pulls 2019 and 2020 Chicago crime data, joins them together, and cleans them
    '''
    
    crimes2019 = get_data_chicago('w98m-zvie')
    crimes2020 = get_data_chicago('qzdf-xmn8')

    crimes = pd.concat([crimes2019, crimes2020])

    crimes = crimes[['ID', 'Date', 'Primary Type', 'FBI Code', 
            'Arrest', 'Community Area', 'Year','Location']]

    return crimes


def community_area():
    ''' 
    Pulls Chicago community area data
    '''
    #Chicago community areas dataset
    comm_areas = get_data_chicago('igwz-8jzy')

    #select only relevant columns
    comm_areas = comm_areas[['COMMUNITY', 'AREA_NUMBE']]
    return comm_areas

def merge_crime_comm(crimes, comm_areas):
    ''' 
    Merges crime and community area data
    '''
    #merge crime df and community areas df
    merged = crimes.merge(comm_areas, left_on="Community Area", 
                          right_on="AREA_NUMBE", how="inner")
    return merged

# run the functions
crimes = crime_data()
comm_areas = community_area()
merged = merge_crime_comm(crimes, comm_areas)
merged.head()

,ID,Date,Primary Type,FBI Code,Arrest,Community Area,Year,Location,COMMUNITY,AREA_NUMBE
0,12182298,12/18/2019 04:51:00 PM,OTHER OFFENSE,26,True,27,2019,"(41.877931634, -87.704810559)",EAST GARFIELD PARK,27
1,11834479,09/19/2019 10:53:00 PM,ROBBERY,03,True,27,2019,"(41.879296357, -87.692055183)",EAST GARFIELD PARK,27
2,11628837,03/08/2019 12:47:00 PM,NARCOTICS,18,True,27,2019,NaN,EAST GARFIELD PARK,27
3,11960598,12/31/2019 08:46:00 AM,NARCOTICS,18,True,27,2019,NaN,EAST GARFIELD PARK,27
4,11892701,11/15/2019 06:12:00 PM,BATTERY,04B,True,27,2019,"(41.870991163, -87.702260725)",EAST GARFIELD PARK,27


In [13]:
count_by_community = merged.groupby(['COMMUNITY','Primary Type']).size().to_frame('COUNT').reset_index()
count_by_community.head()

,COMMUNITY,Primary Type,COUNT
0,ALBANY PARK,ARSON,8
1,ALBANY PARK,ASSAULT,310
2,ALBANY PARK,BATTERY,683
3,ALBANY PARK,BURGLARY,176
4,ALBANY PARK,CONCEALED CARRY LICENSE VIOLATION,1


### Plotting bar graph

In [58]:
def plot_crimes(df, neighborhood):
    df = df[df['COMMUNITY']==neighborhood]
    df = df.sort_values('COUNT', ascending=False).head(10)
    
    barplot = figure(title='{} Crimes'.format(neighborhood), # set up Figure object
                  x_axis_label='Crime', 
                  y_axis_label='Count',
                  x_range=df['Primary Type'],
                  y_range=(0,600),
                  plot_width=800, 
                  plot_height=600)

    barplot.vbar(x=df['Primary Type'], top=df['COUNT'],width=0.9)
    barplot.xaxis.major_label_orientation = "vertical"
    barplot.y_range.start = 0
    
    
    show(barplot)
    
plot_crimes(count_by_community, 'BRIDGEPORT')


## Adding Interactivity with Decorators


**What is a decorator?**

Decorators can add functionality to an existing code.

In [69]:
# Example from https://www.datacamp.com/community/tutorials/decorators-python
def uppercase_decorator(function):
    def wrapper():
        func = function()
        make_uppercase = func.upper()
        return make_uppercase

    return wrapper

In [70]:
@uppercase_decorator
def say_hi():
    return 'hello there'

say_hi()

'HELLO THERE'

### Widgets
https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html

### Adding tabs

In [64]:
neighborhoods = merged['COMMUNITY'].unique()

def plot_crimes2(df, neighborhood):
    df = df[df['COMMUNITY']==neighborhood]
    df = df.sort_values('COUNT', ascending=False).head(10)
    
    # bar plot
    barplot = figure(title='{} Crimes'.format(neighborhood), # set up Figure object
                  x_axis_label='Crime', 
                  y_axis_label='Count',
                  x_range=df['Primary Type'])

    barplot.vbar(x=df['Primary Type'], top=df['COUNT'],width=0.9)
    barplot.xaxis.major_label_orientation = "vertical"
    bar_panel = Panel(child=barplot, title='Bar plot')
    
    # line plot
    lineplot = figure(title='{} Crimes'.format(neighborhood),
                  x_axis_label='Crime', 
                  y_axis_label='Count',
                  x_range=df['Primary Type'])

    lineplot.line(df['Primary Type'], df['COUNT'])
    lineplot.xaxis.major_label_orientation = "vertical"
    line_panel = Panel(child=lineplot, title='Line plot')
    
    # scatter plot
    scatterplot = figure(title='{} Crimes'.format(neighborhood),
                  x_axis_label='Crime', 
                  y_axis_label='Count',
                  x_range=df['Primary Type'])

    scatterplot.circle(df['Primary Type'], df['COUNT'])
    scatterplot.xaxis.major_label_orientation = "vertical"
    scatter_panel = Panel(child=scatterplot, title='Scatter plot')

    tabs = Tabs(tabs=[bar_panel, line_panel, scatter_panel])
    return tabs
    

@interact(neighborhood=neighborhoods)
def make_plot_for(neighborhood='BRIDGEPORT'):
    tabs = plot_crimes2(count_by_community, neighborhood)
    show(tabs)